In [139]:
import respy as rp
import pandas as pd
import numpy as np

In [140]:
params, options, data = rp.get_example_model("kw_94_two")

In [184]:
choices = ["a", "b", "c", "edu", "home"]

cov = np.tile("fsdklfjldskjfldsjlfksakjf", (5, 5))

for i, row in enumerate(choices):
    for j, column in enumerate(choices):
        if row == column:
            cov[i, j] = f"sd_{row}"
        else:
            str_ =f"corr_{row}_{column}"

            cov[i, j] = str_

list(cov[np.tril_indices(5, -1)].flatten())

['corr_b_a',
 'corr_c_a',
 'corr_c_b',
 'corr_edu_a',
 'corr_edu_b',
 'corr_edu_c',
 'corr_home_a',
 'corr_home_b',
 'corr_home_c',
 'corr_home_edu']

In [185]:
list(np.diag(cov)) + list(cov[np.tril_indices(5, -1)].flatten())

['sd_a',
 'sd_b',
 'sd_c',
 'sd_edu',
 'sd_home',
 'corr_b_a',
 'corr_c_a',
 'corr_c_b',
 'corr_edu_a',
 'corr_edu_b',
 'corr_edu_c',
 'corr_home_a',
 'corr_home_b',
 'corr_home_c',
 'corr_home_edu']